# Volumetric heat maps

In [1]:
TRIALS = 100
n = 6
Depth = 6

In [2]:
from collections import Counter
import time
import random
import math
import numpy as np

In [3]:
%env QRACK_QUNITMULTI_DEVICES 1
from pyqrack import QrackSimulator, QrackCircuit

env: QRACK_QUNITMULTI_DEVICES=1


In [4]:
def mcx(circ, c, q):
    circ.ucmtrx([c], [0, 1, 1, 0], q, 1)

def mcy(circ, c, q):
    circ.ucmtrx([c], [0, -1j, 1j, 0], q, 1)

def mcz(circ, c, q):
    circ.ucmtrx([c], [1, 0, 0, -1], q, 1)

def macx(circ, c, q):
    circ.ucmtrx([c], [0, 1, 1, 0], q, 0)

def macy(circ, c, q):
    circ.ucmtrx([c], [0, -1j, 1j, 0], q, 0)

def macz(circ, c, q):
    circ.ucmtrx([c], [1, 0, 0, -1], q, 0)

def rand_u3(circ, q):
    th = random.uniform(0, 4 * math.pi)
    ph = random.uniform(0, 4 * math.pi)
    lm = random.uniform(0, 4 * math.pi)

    c = math.cos(th / 2)
    s = math.sin(th / 2)

    op = []
    op.append(c)
    op.append(-np.exp(1j * lm) * s)
    op.append(np.exp(1j * ph) * s)
    op.append(np.exp(1j * (ph + lm)) * c)

    circ.mtrx(op, q)

sqrt1_2 = 1 / math.sqrt(2)
    
GateCount1Qb = 7
GateCountMultiQb = 12
GateCount2Qb = 6

qubits = []
for i in range(n):
    qubits.append(i)

for trial in range(TRIALS):
    testCase = QrackSimulator(n)

    gate1QbRands = []
    gateMultiQbRands = []

    for d in range(Depth):
        layer1QbRands = []
        for i in range(n):
            gate = random.randint(0, GateCount1Qb)
            layer1QbRands.append(gate)
        gate1QbRands.append(layer1QbRands)

        unusedBits = []
        for i in range(n):
            unusedBits.append(i)

        layerMultiQbRands = []
        while len(unusedBits) > 1:
            b1 = random.choice(unusedBits)
            unusedBits.remove(b1)
            b2 = random.choice(unusedBits)
            unusedBits.remove(b2)

            maxGates = 0
            if len(unusedBits) > 0:
                maxGates = GateCountMultiQb
            else:
                maxGates = GateCount2Qb

            gate = random.randint(0, maxGates)

            b3 = 0
            if gate > GateCount2Qb:
                b3 = random.choice(unusedBits)
                unusedBits.remove(b3)

            multiGate = { 'gate' : gate, 'b1': b1, 'b2': b2, 'b3': b3}
            layerMultiQbRands.append(multiGate)

        gateMultiQbRands.append(layerMultiQbRands)


    randPerm = random.randint(0, (1 << n) - 1)

    circuit = QrackCircuit()
    
    for i in range(n):
        if (randPerm >> i) & 1 == 1:
            circuit.mtrx([ 0, 1, 1, 0 ], i)

    for d in range(Depth):
        layer1QbRands = gate1QbRands[d]
        for i in range(len(layer1QbRands)):
            gate1Qb = layer1QbRands[i]
            if gate1Qb == 0:
                circuit.mtrx([ sqrt1_2, sqrt1_2, sqrt1_2, -sqrt1_2 ], i)
            elif gate1Qb == 1:
                circuit.mtrx([ 0, 1, 1, 0 ], i)
            elif gate1Qb == 2:
                circuit.mtrx([ 0, -1j, 1j, 0 ], i)
            elif gate1Qb == 3:
                circuit.mtrx([ 1, 0, 0, -1 ], i)
            elif gate1Qb == 4:
                circuit.mtrx([ 1, 0, 0, 1j ], i)
            elif gate1Qb == 5:
                circuit.mtrx([ 1, 0, 0, -1j ], i)
            elif gate1Qb == 6:
                circuit.mtrx([ 1, 0, 0, sqrt1_2 + (sqrt1_2 * 1j) ], i)
            else:
                circuit.mtrx([ 1, 0, 0, sqrt1_2 - (sqrt1_2 * 1j) ], i)

        layerMultiQbRands = gateMultiQbRands[d];
        for i in range(len(layerMultiQbRands)):
            multiGate = layerMultiQbRands[i]
            b1 = multiGate['b1']
            b2 = multiGate['b2']
            b3 = multiGate['b3']
            gate = multiGate['gate']
            control = [ b1 ]
            controls = [ b1, b2 ]
            if gate == 0:
                circuit.swap(b1, b2)
            elif gate == 1:
                circuit.ucmtrx(control, [ 0, 1, 1, 0 ], b2, 1)
            elif gate == 2:
                circuit.ucmtrx(control, [ 0, -1j, 1j, 0 ], b2, 1)
            elif gate == 3:
                circuit.ucmtrx(control, [ 1, 0, 0, -1 ], b2, 1)
            elif gate == 4:
                circuit.ucmtrx(control, [ 0, 1, 1, 0 ], b2, 0)
            elif gate == 5:
                circuit.ucmtrx(control, [ 0, -1j, 1j, 0 ], b2, 0)
            elif gate == 6:
                circuit.ucmtrx(control, [ 1, 0, 0, -1 ], b2, 0)
            elif gate == 7:
                circuit.ucmtrx(controls, [ 0, 1, 1, 0 ], b3, 3)
            elif gate == 8:
                circuit.ucmtrx(controls, [ 0, -1j, 1j, 0 ], b3, 3)
            elif gate == 9:
                circuit.ucmtrx(controls, [ 1, 0, 0, -1 ], b3, 3)
            elif gate == 10:
                circuit.ucmtrx(controls, [ 0, 1, 1, 0 ], b3, 0)
            elif gate == 11:
                circuit.ucmtrx(controls, [ 0, -1j, 1j, 0 ], b3, 0)
            else:
                circuit.ucmtrx(controls, [ 1, 0, 0, -1 ], b3, 0)

    circuit.run(testCase)

    # Mirror the circuit
    for d in reversed(range(0, Depth)):
        layerMultiQbRands = gateMultiQbRands[d]
        for i in reversed(range(len(layerMultiQbRands))):
            multiGate = layerMultiQbRands[i]
            b1 = multiGate['b1']
            b2 = multiGate['b2']
            b3 = multiGate['b3']
            gate = multiGate['gate']
            control = [ b1 ]
            controls = [ b1, b2 ]
            if gate == 0:
                testCase.swap(b1, b2)
            elif gate == 1:
                testCase.mcx(control, b2)
            elif gate == 2:
                testCase.mcy(control, b2)
            elif gate == 3:
                testCase.mcz(control, b2)
            elif gate == 4:
                testCase.macx(control, b2)
            elif gate == 5:
                testCase.macy(control, b2)
            elif gate == 6:
                testCase.macz(control, b2)
            elif gate == 7:
                testCase.mcx(controls, b3)
            elif gate == 8:
                testCase.mcy(controls, b3)
            elif gate == 9:
                testCase.mcz(controls, b3)
            elif gate == 10:
                testCase.macx(controls, b3)
            elif gate == 11:
                testCase.macy(controls, b3)
            else:
                testCase.macz(controls, b3)

        layer1QbRands = gate1QbRands[d]
        for i in reversed(range(len(layer1QbRands))):
            gate1Qb = layer1QbRands[i]
            if gate1Qb == 0:
                testCase.h(i)
            elif gate1Qb == 1:
                testCase.x(i)
            elif gate1Qb == 2:
                testCase.y(i)
            elif gate1Qb == 3:
                testCase.z(i)
            elif gate1Qb == 4:
                testCase.adjs(i)
            elif gate1Qb == 5:
                testCase.s(i)
            elif gate1Qb == 6:
                testCase.adjt(i)
            else:
                testCase.t(i)

    result = dict(Counter(testCase.measure_shots(qubits, 100)))

    for key in result:
        if key != randPerm:
            print("Failed!")
print("Done checking.")

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_Gen9_HD_Graphics_NEO.ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_2070_Super.ir
Done checking.
